# Z3 SMT Solver Tutorial
Z3 is a theorem prover from Microsoft Research. It is licensed under the MIT license.

An important application of SMT solvers is formal verification (eg. program symbolic execution, analysis and testing), aimed particularly at finding security vulnerabilities. Important actively-maintained tools in this category include SAGE from Microsoft Research, KLEE, S2E, and Triton. SMT solvers that are particularly useful for symbolic-execution applications include Z3, STP, Z3str2, and Boolector.

In this tutorial we'll show examples for solving satisfiability problems using the Z3 Python bindings.

### Some general background:
* https://en.wikipedia.org/wiki/Boolean_satisfiability_problem
* https://en.wikipedia.org/wiki/First-order_logic
* https://en.wikipedia.org/wiki/Satisfiability_modulo_theories
* https://en.wikipedia.org/wiki/Answer_set_programming

### GitHub repository:
* https://github.com/Z3Prover/z3

### Book with many (!) worked examples (mostly in Python) using Z3:
* https://yurichev.com/writings/SAT_SMT_by_example.pdf

### Reverse engineering demo using Cutter (Radare GUI) and solution validation using Z3
* https://www.youtube.com/watch?v=oNGLIkSUgQo

# Toy example #1 - consecutive integer sum
In this example we are asking:
* Given an integer N
* Is it possible to find n consecutive integers
* So that sum[Int(0), ..., Int(n-1)] == N

In [19]:
import z3 as z

Define our goals

In [20]:
n = 3
N = 1050

Generate n symbolic representations of integers

In [21]:
ints = [z.Int(f'int{i}') for i in range(n)]
ints

[int0, int1, int2]

Instatiate our SMT solver and set some constraints:
* Int[0], ..., Int[n-1] E Int > 0 - integers are natural
* Int[i+1] == Int[i] + 1 - integers must be consecutive
* Sum(Int[0], ..., Int[n-1]) == N - set a constraint on the sum of integers

In [22]:
s = z.Solver()

In [23]:
for i in range(n-1):
    s.add(ints[i] > 0)
    s.add(ints[i+1] == ints[i] + 1)

In [24]:
s.add(z.Sum(ints) == N)

Check satisfiability of our constraint:

In [25]:
if s.check() == z.sat:
    m = s.model()
    print(f'({n} terms) {m[ints[0]].as_long()} + ... + {m[ints[n -1]].as_long()} == {N}')
else:
    print(f'Finding {n} consecutive integers with a sum of {N} is unsatasfiable')

(3 terms) 349 + ... + 351 == 1050


### Let's wrap our model in a function

In [26]:
def check_satisfiability(n: int, N: int, positive: bool = True):
    ints = [z.Int(f'int{i}') for i in range(n)]
    
    s = z.Solver()
    
    for i in range(n-1):
        s.add(ints[i+1] == ints[i] + 1)
        if positive:
            s.add(ints[i] > 0)
        else:
            s.add(ints[i] < 0)
            
    s.add(z.Sum(ints) == N)
    
    return s.check()

In [27]:
for i in range(3, 10):
    print(f'Sum 1337 is {check_satisfiability(n = i, N = 1337)} with {i} integers')

Sum 1337 is unsat with 3 integers
Sum 1337 is unsat with 4 integers
Sum 1337 is unsat with 5 integers
Sum 1337 is unsat with 6 integers
Sum 1337 is sat with 7 integers
Sum 1337 is unsat with 8 integers
Sum 1337 is unsat with 9 integers


# Toy example #2 - cracking the Linear Congruential Generator (LCG)
In this example we'll:
* Take a look at the assembly code for rand() from MSVC 2013
* Calculate the n+1 random number after seeing n numbers generated by rand()
* Calculate the number that was generated before our n numbers 
* Find a seed for rand() that results in 8 consequtive 0's being generated

### Implementation of rand() in MSVC 2013
This is a simplified implementation but it's instructive:
    static unsigned long int next = 1;

    int rand(void)  /* RAND_MAX assumed to be 32767. */
    {
        next = next * 214013 + 2531011;
        return (unsigned)(next/65536) % 32768;
    }

This code snippet produces the following assembly code:

    0x0040112E rand proc near
    0x0040112E call __getptd
    ; Multiply seed (or previous number) by 214013
    0x00401133 imul ecx, [eax+0x14], 214013
    ; Add 2531011
    0x0040113A add ecx, 2531011
    0x00401140 mov [eax+14h], ecx
    ; Shift right 16 bits(i.e. divide by 0d65536) 
    0x00401143 shr ecx, 16
    ; Take modulus of 32768 
    ; This is the same as doing 'eax and 32768 - 1' (eax and 0x7FFF)
    0x00401146 and ecx, 7FFFh
    ; Output
    0x0040114C mov eax, ecx
    0x0040114E retn
    0x0040114E rand endp
    
Now let's assume we ran this rand() 10 times and got the following set of numbers:
37, 29, 74, 95, 98, 40, 23, 58, 61, 17

Based on:
* This series of random numbers 
* Implementation of rand() listed above

Can we calculate the eleventh random number?

### Building the constraints for the problem
In this case we'll be using bit vectors (BitVec) of length 32 so we can do some bit shifting later

In [28]:
output_prev = z.BitVec('output_prev', 32)
states = {f'state{i}': z.BitVec(f'state{i}', 32) for i in range(1, 11)}
output_next = z.BitVec('output_next', 32)

print(type(states['state1']))

<class 'z3.z3.BitVecRef'>


Instantiate the solver

In [29]:
s = z.Solver()

Now let's add the problem constraints. 

Our 1st constraint is that the 10 numbers were acquired by consecutive runs of rand(), therefore:

    state2 == state1 * 214013 + 2531011
    state3 == state2 * 214013 + 2531011
    ...
    state10 == state9 * 214013 + 2531011

In [30]:
for i in range(2, 11):
    s.add(states[f'state{i}'] == states[f'state{i - 1}'] * 214013 + 2531011)

Our 2nd constraint is the values we got for the 10 numbers.
Specifically, based on the assembly code:
* each state is divided by 65536 (shifted 16 bits to the right) 
* then we take reminder of dividing it by 32768 (same as doing n and 0x7FFF, n % 32768)
* this reminder should be equal for each number in our set

Therefore: 


    (state1 >> 16) % 32768 = 37
    ...
    (state10 >> 16) % 32768 = 17

In [31]:
random_nums = [37, 29, 74, 95, 98, 40, 23, 58, 61, 17]

for i in range(2, 10):
    s.add(z.URem((states[f'state{i}'] >> 16) & 0x7FFF ,100) == random_nums[i - 1])

Lastly, we'll set the constraints for the next and previous number in the series:

In [32]:
s.add(output_prev == z.URem((states['state1'] >> 16) & 0x7FFF ,100))
s.add(output_next == z.URem((states['state10'] >> 16) & 0x7FFF ,100))

In [33]:
print(s)

[state2 == state1*214013 + 2531011,
 state3 == state2*214013 + 2531011,
 state4 == state3*214013 + 2531011,
 state5 == state4*214013 + 2531011,
 state6 == state5*214013 + 2531011,
 state7 == state6*214013 + 2531011,
 state8 == state7*214013 + 2531011,
 state9 == state8*214013 + 2531011,
 state10 == state9*214013 + 2531011,
 URem(state2 >> 16 & 32767, 100) == 29,
 URem(state3 >> 16 & 32767, 100) == 74,
 URem(state4 >> 16 & 32767, 100) == 95,
 URem(state5 >> 16 & 32767, 100) == 98,
 URem(state6 >> 16 & 32767, 100) == 40,
 URem(state7 >> 16 & 32767, 100) == 23,
 URem(state8 >> 16 & 32767, 100) == 58,
 URem(state9 >> 16 & 32767, 100) == 61,
 output_prev == URem(state1 >> 16 & 32767, 100),
 output_next == URem(state10 >> 16 & 32767, 100)]


In [34]:
print(s.check())

sat


In [35]:
print(s.model())

[state3 = 2276903645,
 state4 = 1467740716,
 state5 = 3163191359,
 state7 = 4108542129,
 state8 = 2839445680,
 state2 = 998088354,
 state6 = 4214551046,
 state1 = 1791599627,
 state9 = 548002995,
 output_next = 17,
 output_prev = 37,
 state10 = 1390515370]


### Minimum number of required values
Next, let's try to figure out what's the minimum number of consecutive values required to figure out the value of the next call to rand().

In order to do that, we'll wrap the logic described above in a function. This function will generate the constraints based on a subset of random_nums and compare the result to the next number in the sequence.

In [44]:
def break_lcg(nums: list, next_num: int):
    n_nums = len(nums)
    #print(f'len nums: {n_nums}')
    
    states = {f'state{i}': z.BitVec(f'state{i}', 32) for i in range(1, n_nums + 2)}
    #print(states)
    output_next = z.BitVec('output_next', 32)
    
    s = z.Solver()
    
    for i in range(2, n_nums + 2):
        s.add(states[f'state{i}'] == states[f'state{i - 1}'] * 214013 + 2531011)
        
    for i in range(1, n_nums + 1):
        s.add(z.URem((states[f'state{i}'] >> 16) & 0x7FFF ,100) == nums[i - 1])
        
    s.add(output_next == z.URem((states[f'state{n_nums + 1}'] >> 16) & 0x7FFF ,100))
    
    #print(s)
    
    if s.check() == z.sat:
        print(f'For the sequence: {nums}, problem is satisfiable')
        print(f'We were expecting: {next_num} and got: {s.model()[output_next]}\n')
    else:
        print(f'For the sequence: {nums}, problem is unsatisfiable')
    
    return s, states

In [37]:
random_nums = [37, 29, 74, 95, 98, 40, 23, 58, 61, 17]

In [38]:
for i in range(3, 10):
    break_lcg(random_nums[:i], random_nums[i])

For the sequence: [37, 29, 74], problem is satisfiable
We were expecting: 95 and got: 45

For the sequence: [37, 29, 74, 95], problem is satisfiable
We were expecting: 98 and got: 29

For the sequence: [37, 29, 74, 95, 98], problem is satisfiable
We were expecting: 40 and got: 40

For the sequence: [37, 29, 74, 95, 98, 40], problem is satisfiable
We were expecting: 23 and got: 23

For the sequence: [37, 29, 74, 95, 98, 40, 23], problem is satisfiable
We were expecting: 58 and got: 58

For the sequence: [37, 29, 74, 95, 98, 40, 23, 58], problem is satisfiable
We were expecting: 61 and got: 61

For the sequence: [37, 29, 74, 95, 98, 40, 23, 58, 61], problem is satisfiable
We were expecting: 17 and got: 17



Notice that based on the results above, even if the constraints are satisfiable, the next number in the sequence is not necessarily the one we expected. Based on these results, it's obvious that when we feed 4 numbers into break_lcg() there are __at least 2 distinct__ internal states that satisfy the constraints. Furthermore, these distinct internal states result in different values for the next number in the sequence.

### Enumerating solutions
Next, we'll try to enumerate all possible solutions that satisfy our constraints. In order to do that, we'll add negation constraints for the solutions we already found. Namely:

    state[1], ..., state[n] != solution[state[1]], ..., solution[state[n]]
    
In the function defined below we iteratively add this constraint every time we find a solution. This is instructive for studying the internal states leading to a specific result.

In [39]:
def count_solutions(nums: list, next_num: int):
    s, states = break_lcg(nums = nums, next_num = next_num)
    
    counter = 0
     
    while s.check() == z.sat:
        counter += 1
        
        print(f'Solution #{counter}')
        print(f'{s.model()}\n')
        or_expression = 'z.Or('
        for i in states.keys():
            if i == 'output_next': continue
            or_expression += f'states[i] != s.model()[states[i]], '
        or_expression += ')'
        
        s.add(eval(or_expression))
        
    print(f'Found a total of {counter} solutions')

In [40]:
count_solutions(random_nums[:5], random_nums[5])

For the sequence: [37, 29, 74, 95, 98], problem is satisfiable
We were expecting: 40 and got: 40

Solution #1
[state3 = 2276903645,
 state4 = 1467740716,
 state1 = 1791599627,
 state2 = 998088354,
 state5 = 3163191359,
 output_next = 40,
 state6 = 4214551046]

Solution #2
[state3 = 129419997,
 state4 = 3615224364,
 state1 = 3939083275,
 state5 = 1015707711,
 output_next = 40,
 state2 = 3145572002,
 state6 = 2067067398]

Found a total of 2 solutions


In [11]:
count_solutions(random_nums[:9], random_nums[9])

For the sequence: [37, 29, 74, 95, 98, 40, 23, 58, 61], problem is satisfiable
We were expecting: 17 and got: 17

Solution #1
[state3 = 2276903645,
 state5 = 3163191359,
 state7 = 4108542129,
 state2 = 998088354,
 state6 = 4214551046,
 state4 = 1467740716,
 state1 = 1791599627,
 state8 = 2839445680,
 state9 = 548002995,
 output_next = 17,
 state10 = 1390515370]

Solution #2
[state3 = 129419997,
 state5 = 1015707711,
 state7 = 1961058481,
 state2 = 3145572002,
 state6 = 2067067398,
 state4 = 3615224364,
 state9 = 2695486643,
 state1 = 3939083275,
 state10 = 3537999018,
 output_next = 17,
 state8 = 691962032]

Found a total of 2 solutions


__Analysis__: feeding the algorithm with either 5 or 9 consecutive rand() numbers we were able to identify 2 distinct internal states satisfying the constraints. While the internal states are distinct, both produce the correct number in the sequence. Furthermore, we've identified two distinct initial internal states (i.e. state1, 3939083275 or 1791599627) that will lead to exactly the same sequence of numbers. We can predict with full confidence next number in the sequence if we've observed at least 5 numbers.

Now let's try the same approach on a sequence of 4 numbers (that we already know does not produce the next number in the sequence).

In [13]:
count_solutions(random_nums[:4], random_nums[4])

For the sequence: [37, 29, 74, 95], problem is satisfiable
We were expecting: 98 and got: 29

Solution #1
[state3 = 2578337618,
 state1 = 1916076056,
 state2 = 3184918139,
 state4 = 1847818445,
 output_next = 29,
 state5 = 2352588892]

Solution #2
[state3 = 3109162427,
 state4 = 3372687762,
 state1 = 874085241,
 state5 = 4004643341,
 output_next = 38,
 state2 = 2601603160]

Solution #3
[state3 = 961678779,
 state4 = 1225204114,
 state1 = 3021568889,
 state5 = 1857159693,
 output_next = 38,
 state2 = 454119512]

Solution #4
[state3 = 987943193,
 state4 = 39047032,
 state1 = 2883972967,
 state5 = 2863599707,
 output_next = 27,
 state2 = 3728813198]

Solution #5
[state3 = 3135426841,
 state4 = 2186530680,
 state1 = 736489319,
 state5 = 716116059,
 output_next = 27,
 state2 = 1581329550]

Solution #6
[state3 = 1053456564,
 state4 = 1978860711,
 state1 = 2477638378,
 state5 = 964619470,
 output_next = 18,
 state2 = 3047259653]

Solution #7
[state3 = 3200940212,
 state4 = 4126344359,
 state1

__Analysis__: feeding the algorithm with either 4 consecutive rand() numbers we were able to identify 50 (!) distinct internal states satisfying the constraints. For referance, these are next number in the sequence predicted by each solution:
     
    29, 38, 38, 27, 27, 18, 18, 21, 21, 18, 18, 54, 54, 63, 10, 61, 61, 77, 82, 10, 63, 26, 26, 4, 7, 82, 83, 83,  88, 88, 49, 49, 85, 85, 4, 62, 90, 90, 7, 7, 62, 46, 46, 98, 98, 62, 62, 29, 93, 93
  

While the internal are be distinct, all the solutions we found must produce the sequence: 37, 29, 74, 95. Meaning we found a set of 50 initial states (state1) that produces this exact sequence. Furthermore, the 50 distinct solutions differ only by the 5th number in the sequence. Specifically, there are 23 distinct values that 5th number can take if we know the first 4 numbers in the sequence. Essentialy, we have found a finite number of possible solutions. As such, we can predict with a confidence of 4% the next number in the sequence, if we've observed 4 numbers.

### Conclusions
Based on the experiment outlined above, we can find a finite number of initial states that produce a specific sequence of (pseudo) random numbers. This statement holds for as long as we've observed a sufficiantly large sequence of consecutive numbers. Furthermore, it seems that for the values used in the MSVC 2013 rand() function (which is known to use relatively weak ones), it's enough to have observed a sequence of 5 numbers in order to predict with full confidence the 6th one. 

One possible extension of the approach outlined above is finding the seed required to produce a sequence of, for example 4 zeros:

In [47]:
count_solutions([0, 0, 0, 0], 0)

For the sequence: [0, 0, 0, 0], problem is satisfiable
We were expecting: 0 and got: 10

Solution #1
[state3 = 1376290615,
 state1 = 478474261,
 state2 = 3399246468,
 state4 = 3818693918,
 output_next = 10,
 state5 = 3766921065]

Solution #2
[state3 = 3884240809,
 state4 = 4290515912,
 state1 = 2658729335,
 state5 = 831226731,
 output_next = 83,
 state2 = 1081360990]

Solution #3
[state3 = 1736757161,
 state4 = 2143032264,
 state1 = 511245687,
 state5 = 2978710379,
 output_next = 83,
 state2 = 3228844638]

Solution #4
[state3 = 3825208236,
 state4 = 2051287999,
 state1 = 2750434338,
 state5 = 808834950,
 output_next = 41,
 state2 = 3438592605]

Solution #5
[state3 = 1677724588,
 state4 = 4198771647,
 state1 = 602950690,
 state5 = 2956318598,
 output_next = 41,
 state2 = 1291108957]

Solution #6
[state3 = 1972677763,
 state4 = 583296314,
 state1 = 1042056705,
 state5 = 4067088149,
 output_next = 90,
 state2 = 1802260672]

Solution #7
[state3 = 2357233349,
 state4 = 314596980,
 state1 = 

In [48]:
count_solutions([0, 0, 0, 0, 0], 0)

For the sequence: [0, 0, 0, 0, 0], problem is unsatisfiable
Found a total of 0 solutions


__Bottom line__: based on these two experiments, there are 38 initial states (seeds) that will result in 4 consecutive zeros, while there is no seed that will produce 5 consecutive zeros.